In [ ]:
#install requirements
%pip install uv
!uv pip install --system langgraph langchain langchain-core langchain-openai langchain-valyu python-dotenv requests google-api-python-client google-auth-httplib2 google-auth-oauthlib



[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Using Python 3.13.2 environment at: /Library/Frameworks/Python.framework/Versions/3.13
Audited 10 packages in 19ms


In [ ]:
#import libraries

import os
import base64
from typing import List, Optional
import getpass
from langchain.chat_models import init_chat_model

from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pathlib import Path
from langchain_core.messages import HumanMessage
from langchain.agents import AgentExecutor

os.environ["OPENAI_API_KEY"] = "FILL"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "FILL"

model = init_chat_model("gpt-4.1")

In [ ]:
#trend tools

@tool 
def brownianModel(TICKER, START_DATE, END_DATE, PRED_END_DATE):
    """ 
    Brownian model for stocks
    """
    # main variables
    stock_name = TICKER
    start_date = START_DATE
    end_date = END_DATE
    pred_end_date = PRED_END_DATE
    scen_size = 10000
    So = train_set.iloc[-1]
    dt = 1  # day

    # -----------------------------
    # Download and prepare data
    # -----------------------------

    prices = yf.download(tickers=stock_name, start=start_date, end=pred_end_date)

    # ---- FIX 1: Robust price column selection ----
    if isinstance(prices.columns, pd.MultiIndex):
        if ('Adj Close', stock_name) in prices.columns:
            prices = prices[('Adj Close', stock_name)]
        else:
            prices = prices[('Close', stock_name)]
    else:
        if 'Adj Close' in prices.columns:
            prices = prices['Adj Close']
        else:
            prices = prices['Close']

    train_set = prices.loc[:end_date]
    test_set = prices.loc[end_date:pred_end_date]
    daily_returns = ((train_set / train_set.shift(1)) - 1)[1:]

    n_of_wkdays = pd.date_range(
        start=pd.to_datetime(end_date) + pd.Timedelta('1 days'),
        end=pd.to_datetime(pred_end_date)
    ).to_series().map(lambda x: 1 if x.isoweekday() in range(1, 6) else 0).sum()

    T = n_of_wkdays
    N = int(T / dt)
    t = np.arange(1, N + 1)

    mu = np.mean(daily_returns)
    sigma = np.std(daily_returns)

    b = {str(scen): np.random.normal(0, 1, N) for scen in range(1, scen_size + 1)}
    W = {str(scen): b[str(scen)].cumsum() for scen in range(1, scen_size + 1)}

    drift = (mu - 0.5 * sigma ** 2) * t
    diffusion = {str(scen): sigma * W[str(scen)] for scen in range(1, scen_size + 1)}

    S = np.array([So * np.exp(drift + diffusion[str(scen)]) for scen in range(1, scen_size + 1)])
    S = np.hstack((np.array([[So] for _ in range(scen_size)]), S))


    S_max = [S[:, i].max() for i in range(0, N)]
    S_min = [S[:, i].min() for i in range(0, N)]
    S_pred = 0.5 * np.array(S_max) + 0.5 * np.array(S_min)

    # Standard Monte Carlo estimator: expected price (mean across simulations)
    #S_pred = np.median(S, axis=0)[1:]



    # ---- FIX 2: Correct final_df construction (Series-safe) ----
    # Align prediction length with available real prices
    min_len = min(len(test_set), len(S_pred))

    final_df = pd.DataFrame({
        'real': test_set.iloc[:min_len].values,
        'pred': S_pred[:min_len]
    }, index=test_set.index[:min_len])

    #mse = np.mean((final_df['pred'] - final_df['real']) ** 2)

    return final_df['pred']






@tool
def mlModel():
    """ 
    RNN model for stock prediction
    """

#noise tools
@tool 
def generalInfo():
    """ 
    information regarding the general market of the stock
    """

@tool 
def specificInfo():
    """ 
    information regarding the stock
    """



In [ ]:
#trend agent
trendAgent = create_openai_functions_agent(
    model,
    tools=[brownianModel, mlModel],
    prompt= FILL,
)

trendAgentExecuter = AgentExecutor(
    agent=trendAgent, 
    tools=[brownianModel, mlModel], 
    verbose=True
)


#noise agent
noiseAgent = create_openai_functions_agent(
    model,
    tools=[generalInfo, specificInfo],
    prompt= FILL,
)

noiseAgentExecuter = AgentExecutor(
    agent=noiseAgent, 
    tools=[generalInfo, specificInfo], 
    verbose=True
)


In [ ]:
@tool 
def trendAgentTool():
    trendAgentExecuter.invoke()

@tool 
def noiseAgentTool():
    noiseAgentExecuter.invoke()

supAgent = create_openai_functions_agent(
    model,
    tools=[trendAgentTool, noiseAgentTool],
    prompt= FILL,
)